# 기상청 지상 관측자료 - 일자료(기간 조회)
+ 작성: 김지훈
+ 수정: 임경호

https://apihub.kma.go.kr/

참고자료 : https://leedakyeong.tistory.com/entry/Python-Crawling-API-%ED%99%9C%EC%9A%A9%ED%95%98%EC%97%AC-%EA%B3%BC%EA%B1%B0-%EA%B8%B0%EC%83%81-%EA%B4%80%EC%B8%A1-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0

In [9]:
import requests
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

# user-defined modules
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

In [10]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT date_id FROM weather ORDER BY date_id DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

In [12]:
# 2023년 1월 1일부터 2023년 6월 30일까지 데이터 일괄 저장
start_date = myutil.get_next_day(last_date)
end_date = myutil.get_previous_day()
print(f'날씨 테이블에는 {last_date}까지 데이터가 저장되어 있으며, 이 날짜 이후인 {start_date}부터 {end_date}까지 날씨 데이터를 가져와야 합니다.')

날씨 테이블에는 20230702까지 데이터가 저장되어 있으며, 이 날짜 이후인 20230703부터 20230716까지 날씨 데이터를 가져와야 합니다.


## API를 활용한 데이터 수집 

In [13]:
# url = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php?tm1=20151211&tm2=20151214&stn=108&help=1&authKey=853B8q9uTi6dwfKvbq4uqQ'
url = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php'
key = "853B8q9uTi6dwfKvbq4uqQ"      #개인 인증키
region = 143                        # STN(ID) 대구 지역코드
# 컬럼 설정: 해당 API 특성 상 사용할 컬럼명을 미리 지정해줘야 사용가능
col_name =["TM","STN","WS_AVG","WR_DAY","WD_MAX","WS_MAX",
           "WS_MAX_TM", "WD_INS", "WS_INS","WS_INS_TM","TA_AVG","TA_MAX",
           "TA_MAX_TM","TA_MIN","TA_MIN_TM", "TD_AVG", "TS_AVG", "TG_MIN",
           "HM_AVG","HM_MIN","HM_MIN_TM", "PV_AVG", "EV_S", "EV_L", 
           "FG_DUR","PA_AVG","PS_AVG", "PS_MAX", "PS_MAX_TM", "PS_MIN",
           "PS_MIN_TM","CA_TOT","SS_DAY", "SS_DUR", "SS_CMB", "SI_DAY",
           "SI_60M_MAX","SI_60M_MAX_TM","RN_DAY", "RN_D99", "RN_DUR", "RN_60M_MAX",
           "RN_60M_MAX_TM","RN_10M_MAX","RN_10M_MAX_TM", "RN_POW_MAX", "RN_POW_MAX_TM", "SD_NEW",
           "SD_NEW_TM","SD_MAX","SD_MAX_TM", "TE_05", "TE_10", "TE_15",
           "TE_30", "TE_50"
           ]
#데이터 수집 시작 날짜와 끝 날짜를 작성 
# start_date = 20230701 #시작 날짜(수정) 
# end_date = 20230702 # 끝 날짜 (수정)

In [14]:
params ={'tm1' : start_date,     # 시작 날짜
         'tm2' : end_date,       # 끝 날짜
         'stn' : region,    # 지역번호
         'help' : 0,        # 0: 설명이 안나옴, 1: 설명이 나옴 
         'authKey' : key}

response = requests.get(url, params=params, verify=False)

In [15]:
if response.status_code == 200:     # URL GET '200 정상'
    text = response.text
    text = text.split("\n")[5:-2] #앞의 필요없는  5줄 부분과 뒤의 필요없는 2줄 부분을 제거
    
    df_weather = pd.DataFrame(text)[0].str.split(expand=True) #데이터프레임 컬럼으로 변경
    df_weather.columns = col_name

In [16]:
df_weather

,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,RN_POW_MAX_TM,SD_NEW,SD_NEW_TM,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50
0,20230703,143,2.1,1823,11,3.9,1154,14,6.5,1031,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
1,20230704,143,2.0,1728,11,4.0,651,18,6.2,1151,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
2,20230705,143,3.0,2630,27,8.4,1347,29,14.6,1219,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
3,20230706,143,1.2,1075,27,3.7,1336,27,5.6,1335,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
4,20230707,143,1.8,1566,11,5.3,1750,11,9.2,1758,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
5,20230708,143,1.7,1489,11,4.5,1819,14,6.7,1740,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
6,20230709,143,1.5,1281,11,2.7,10,14,5.0,335,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
7,20230710,143,1.6,1360,25,4.2,1651,25,8.1,1650,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
8,20230711,143,1.2,1056,23,8.3,1411,23,18.8,1404,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
9,20230712,143,1.3,1082,25,3.6,1546,25,6.2,1545,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0


### 데이터 전처리

In [17]:
#데이터 전처리 
df_weather= df_weather[['TM','TA_AVG','TA_MAX','TA_MIN']] #필요한 부분만 추출>|날짜|평균기온|최고기온|최저기온|
df_weather.columns = ['date_id','ta_avg','ta_max','ta_min'] #컬럼명 변경 > SQL과 똑같이 설정

#타입 변경
df_weather['ta_avg'] = df_weather['ta_avg'].astype(float) #실수형으로 변경
df_weather['ta_max'] = df_weather['ta_max'].astype(float) #실수형으로 변경
df_weather['ta_min'] = df_weather['ta_min'].astype(float) #실수형으로 변경

#기온 편차 컬럼 생성
df_weather['ta_diff'] = df_weather['ta_max']-df_weather['ta_min'] 
df_weather

,date_id,ta_avg,ta_max,ta_min,ta_diff
0,20230703,27.1,32.5,22.5,10.0
1,20230704,24.9,29.0,23.3,5.7
2,20230705,27.5,32.6,23.4,9.2
3,20230706,27.5,34.4,20.7,13.7
4,20230707,21.9,25.7,18.0,7.7
5,20230708,23.1,27.2,19.8,7.4
6,20230709,26.6,31.2,21.8,9.4
7,20230710,28.3,33.8,24.1,9.7
8,20230711,24.8,30.0,22.9,7.1
9,20230712,27.3,32.4,23.1,9.3


## mdw.weather 테이블에 데이터 저장

In [18]:
import datetime as dt
dt_now = dt.datetime.now()
dt_now_str = dt_now.strftime('%Y-%m-%d %H:%M:%S')

user_name = 'lkh'

In [19]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_weather.itertuples():
    sql = "insert into weather (date_id, ta_avg, ta_max, ta_min, ta_diff, create_datetime, create_id) \
                        values (%s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], dt_now_str, user_name))
conn.commit()
conn.close()